In [1]:
from preprocess import load_data
import numpy as np
import RegularizedLogisticRegression
from random import randint
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.ensemble import GradientBoostingClassifier
#Using 7 baseline methods to predict and compute results' AUC.
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import  MLPClassifier
import glob
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

import importlib
import preprocess
importlib.reload(RegularizedLogisticRegression)

<module 'RegularizedLogisticRegression' from '/Users/calebziems/Documents/SCHOOL/Machine Learning/breast-cancer-prediction/src/RegularizedLogisticRegression.py'>

In [2]:
key = ['CELLULARITY_0', 'CELLULARITY_1', 'CELLULARITY_2', 'CELLULARITY_3', 
       'CHEMOTHERAPY_0', 'CHEMOTHERAPY_1', 
       'COHORT_1', 'COHORT_2', 'COHORT_3', 'COHORT_4', 'COHORT_5',
       'ER_IHC_0', 'ER_IHC_1', 'ER_IHC_2',
       'HER2_SNP6_0', 'HER2_SNP6_1', 'HER2_SNP6_2', 'HER2_SNP6_3',
       'HORMONE_THERAPY_0', 'HORMONE_THERAPY_1',
       'INFERRED_MENOPAUSAL_STATE_0', 'INFERRED_MENOPAUSAL_STATE_1',
       'INTCLUST_0', 'INTCLUST_1', 'INTCLUST_2', 'INTCLUST_3', 'INTCLUST_4', 'INTCLUST_5', 'INTCLUST_6', 'INTCLUST_7', 'INTCLUST_8', 'INTCLUST_9', 'INTCLUST_10',
       'CLAUDIN_SUBTYPE_0', 'CLAUDIN_SUBTYPE_1', 'CLAUDIN_SUBTYPE_2', 'CLAUDIN_SUBTYPE_3', 'CLAUDIN_SUBTYPE_4', 'CLAUDIN_SUBTYPE_5', 'CLAUDIN_SUBTYPE_6', 
       'THREEGENE_0', 'THREEGENE_1', 'THREEGENE_2', 'THREEGENE_3', 'THREEGENE_4',
       'LATERALITY_0', 'LATERALITY_1', 'LATERALITY_2',
       'RADIO_THERAPY_0', 'RADIO_THERAPY_1',
       'HISTOLOGICAL_SUBTYPE_0', 'HISTOLOGICAL_SUBTYPE_1', 'HISTOLOGICAL_SUBTYPE_2', 'HISTOLOGICAL_SUBTYPE_3', 'HISTOLOGICAL_SUBTYPE_4', 'HISTOLOGICAL_SUBTYPE_5', 'HISTOLOGICAL_SUBTYPE_6', 'HISTOLOGICAL_SUBTYPE_7', 'HISTOLOGICAL_SUBTYPE_8',
       'BREAST_SURGERY_0', 'BREAST_SURGERY_1', 'BREAST_SURGERY_2',
       'LYMPH_NODES_EXAMINED_POSITIVE', 'NPI', 'AGE_AT_DIAGNOSIS'
      ]

In [3]:
Ctr_X, Ctr_Y, Cval_X, Cval_Y, Ct_X, Ct_Y, Gtr_X, Gtr_Y, Gval_X, Gval_Y, Gt_X, Gt_Y = preprocess.load_data(False)

In [4]:
C_X = np.concatenate([Ctr_X, Cval_X])
C_Y = np.concatenate([Ctr_Y, Cval_Y])

G_X = np.concatenate([Gtr_X, Gval_X])
G_Y = np.concatenate([Gtr_Y, Gval_Y])

In [5]:
def roc_score(clf, type, X, Y, X_val, Y_val):
    clf.fit(X, Y)
    y_pred = clf.predict(X_val)
    fpr, tpr, thresholds = roc_curve(Y_val, y_pred)
    a = auc(fpr, tpr)
    print(type, "ROC AUC: ", a)
    return a

# KNN

In [6]:
scores = np.zeros(5)
for i, filename in enumerate(glob.glob('CV/KNN/*clinical*.txt')):
    with open(filename, 'r') as file:
        p_star, p_opt = file.readline().split()
        clf = KNeighborsClassifier(n_neighbors=int(p_star))
        scores[i] = roc_score(clf, 'Clinical (validation) k='+p_star, Ctr_X, Ctr_Y, Cval_X, Cval_Y)
        
print("average:", np.mean(scores))

Clinical (validation) k=19 ROC AUC:  0.5868191721132898
Clinical (validation) k=57 ROC AUC:  0.6061002178649237
Clinical (validation) k=12 ROC AUC:  0.5825708061002179
Clinical (validation) k=25 ROC AUC:  0.6044662309368193
Clinical (validation) k=41 ROC AUC:  0.60359477124183
average: 0.5967102396514161


In [7]:
clf = KNeighborsClassifier(n_neighbors=57)
roc_score(clf, 'Clinical (test) k='+p_star, C_X, C_Y, Ct_X, Ct_Y)

Clinical (test) k=41 ROC AUC:  0.5581102722489133


0.5581102722489133

In [8]:
scores = np.zeros(5)
for i, filename in enumerate(glob.glob('CV/KNN/*genomic*.txt')):
    with open(filename, 'r') as file:
        p_star, p_opt = file.readline().split()
        clf = KNeighborsClassifier(n_neighbors=int(p_star))
        scores[i] = roc_score(clf, 'Genomic (validation) k='+p_star, Gtr_X, Gtr_Y, Gval_X, Gval_Y)
print("average:", np.mean(scores))

Genomic (validation) k=21 ROC AUC:  0.5
Genomic (validation) k=7 ROC AUC:  0.5218954248366013
Genomic (validation) k=99 ROC AUC:  0.5
Genomic (validation) k=99 ROC AUC:  0.5
Genomic (validation) k=3 ROC AUC:  0.5118736383442265
average: 0.5067538126361655


In [9]:
clf = KNeighborsClassifier(n_neighbors=7)
roc_score(clf, 'Genomic (test) k=7', G_X, G_Y, Gt_X, Gt_Y)

Genomic (test) k=7 ROC AUC:  0.5210478151452757


0.5210478151452757

# SGD, alpha=0.95

In [10]:
scores = np.zeros(5)
for i, filename in enumerate(glob.glob('CV/SGD_0.95/*clinical*.txt')):
    with open(filename, 'r') as file:
        p_star, p_opt = file.readline().split()
        clf = SGDClassifier(alpha=float(p_star), loss='log', penalty='elasticnet', l1_ratio=0.95, shuffle=False, random_state=43)
        scores[i] = roc_score(clf, 'Clinical (validation) lambda='+p_star, Ctr_X, Ctr_Y, Cval_X, Cval_Y)
print("average:", np.mean(scores))

Clinical (validation) lambda=0.007663410868007455 ROC AUC:  0.7055555555555556
Clinical (validation) lambda=0.02582618760682675 ROC AUC:  0.6313725490196078
Clinical (validation) lambda=0.02582618760682675 ROC AUC:  0.6313725490196078
Clinical (validation) lambda=0.049370478528390035 ROC AUC:  0.6557734204793028
Clinical (validation) lambda=0.012458833642950081 ROC AUC:  0.7004357298474946
average: 0.6649019607843137


In [11]:
clf = SGDClassifier(alpha=0.012458833642950081, loss='log', penalty='elasticnet', l1_ratio=0.95, shuffle=False)
roc_score(clf, 'Clinical (test) lambda=0.0124', C_X, C_Y, Ct_X, Ct_Y)

Clinical (test) lambda=0.0124 ROC AUC:  0.6482498284145505


0.6482498284145505

In [12]:
scores = np.zeros(5)
for i, filename in enumerate(glob.glob('CV/SGD_0.95/*genomic*.txt')):
    with open(filename, 'r') as file:
        p_star, p_opt = file.readline().split()
        clf = SGDClassifier(alpha=float(p_star), loss='log', penalty='elasticnet', l1_ratio=0.95, shuffle=False, random_state=43)
        scores[i] = roc_score(clf, 'Genomic (validation) lambda='+p_star, Gtr_X, Gtr_Y, Gval_X, Gval_Y)
print("average:", np.mean(scores))

Genomic (validation) lambda=0.0015167168884709225 ROC AUC:  0.6056644880174292
Genomic (validation) lambda=0.002097046401323233 ROC AUC:  0.6072984749455337
Genomic (validation) lambda=0.00651733960488242 ROC AUC:  0.6014161220043572
Genomic (validation) lambda=0.005111433483440166 ROC AUC:  0.5837690631808279
Genomic (validation) lambda=0.0036969127071950283 ROC AUC:  0.6056644880174292
average: 0.6007625272331154


In [13]:
clf = SGDClassifier(alpha=0.002097046401323233, loss='log', penalty='elasticnet', l1_ratio=0.95, shuffle=False, random_state=43)
roc_score(clf, 'Genomic (test) lambda=0.00209', G_X, G_Y, Gt_X, Gt_Y)

Genomic (test) lambda=0.00209 ROC AUC:  0.6196522534889042


0.6196522534889042

# Linear SVC

In [14]:
scores = np.zeros(5)
for i, filename in enumerate(glob.glob('CV/SVC_linear/*clinical*.txt')):
    with open(filename, 'r') as file:
        p_star, p_opt = file.readline().split()
        clf = SVC(kernel='linear', random_state=0, C=float(p_star))
        scores[i] = roc_score(clf, 'Clinical (validation) C='+p_star, Ctr_X, Ctr_Y, Cval_X, Cval_Y)
print('average:', np.mean(scores))

Clinical (validation) C=0.0093260334688322 ROC AUC:  0.6742919389978214
Clinical (validation) C=0.004862601580065354 ROC AUC:  0.6465141612200436
Clinical (validation) C=0.004862601580065354 ROC AUC:  0.6465141612200436
Clinical (validation) C=0.010974987654930556 ROC AUC:  0.6742919389978214
Clinical (validation) C=0.01788649529057435 ROC AUC:  0.7113289760348583
average: 0.6705882352941176


In [ ]:
clf = SVC(kernel='linear', random_state=0, C=0.01788649529057435)
roc_score(clf, 'Clinical (test) C=0.0179', C_X, C_Y, Ct_X, Ct_Y)

Clinical (test) C=0.0179 ROC AUC:  0.659116906886296


0.659116906886296

In [6]:
scores = np.zeros(5)
for i, filename in enumerate(glob.glob('CV/SVC_linear/*genomic*.txt')):
    with open(filename, 'r') as file:
        p_star, p_opt = file.readline().split()
        clf = SVC(kernel='linear', random_state=0, C=float(p_star))
        scores[i] = roc_score(clf, 'Genomic (validation) C='+p_star, Gtr_X, Gtr_Y, Gval_X, Gval_Y)
print('average:', np.mean(scores))

Genomic (validation) C=2.559547922699533e-05 ROC AUC:  0.6238562091503268
Genomic (validation) C=1.2648552168552958e-05 ROC AUC:  0.583442265795207
Genomic (validation) C=1.5998587196060572e-05 ROC AUC:  0.5893246187363834
Genomic (validation) C=3.237457542817647e-05 ROC AUC:  0.6272331154684095
Genomic (validation) C=1.2648552168552958e-05 ROC AUC:  0.583442265795207
average: 0.6014596949891067


In [7]:
clf = SVC(kernel='linear', random_state=0, C=3.237457542817647e-05)
roc_score(clf, 'Genomic (test) C=3.237', G_X, G_Y, Gt_X, Gt_Y)

Genomic (test) C=3.237 ROC AUC:  0.6054678563257835


0.6054678563257835

# RBF SVC

In [8]:
scores = np.zeros(5)
for i, filename in enumerate(glob.glob('CV/SVC_rbf/*clinical*.txt')):
    with open(filename, 'r') as file:
        p_star, p_opt = file.readline().split()
        clf = SVC(kernel='rbf', random_state=0, C=float(p_star))
        scores[i] = roc_score(clf, 'Clinical (validation) C='+p_star, Ctr_X, Ctr_Y, Cval_X, Cval_Y)
print('average:', np.mean(scores))

Clinical (validation) C=1.0985411419875584 ROC AUC:  0.6877995642701525
Clinical (validation) C=0.8685113737513529 ROC AUC:  0.6625272331154685
Clinical (validation) C=0.5428675439323859 ROC AUC:  0.6498910675381263
Clinical (validation) C=1.757510624854793 ROC AUC:  0.7004357298474946
Clinical (validation) C=0.8685113737513529 ROC AUC:  0.6625272331154685
average: 0.6726361655773421


In [9]:
clf = SVC(kernel='rbf', random_state=0, C=1.757510624854793)
roc_score(clf, 'Clinical (test) C=1.757', C_X, C_Y, Ct_X, Ct_Y)

Clinical (test) C=1.757 ROC AUC:  0.6482498284145505


0.6482498284145505

In [10]:
scores = np.zeros(5)
for i, filename in enumerate(glob.glob('CV/SVC_rbf/*genomic*.txt')):
    with open(filename, 'r') as file:
        p_star, p_opt = file.readline().split()
        clf = SVC(kernel='rbf', random_state=0, C=float(p_star))
        scores[i] = roc_score(clf, 'Genomic (validation) C='+p_star, Gtr_X, Gtr_Y, Gval_X, Gval_Y)
print('average:', np.mean(scores))

Genomic (validation) C=1.3894954943731375 ROC AUC:  0.6019607843137255
Genomic (validation) C=1.3894954943731375 ROC AUC:  0.6019607843137255
Genomic (validation) C=2.2229964825261956 ROC AUC:  0.6213507625272332
Genomic (validation) C=1.3894954943731375 ROC AUC:  0.6019607843137255
Genomic (validation) C=1.0985411419875584 ROC AUC:  0.5985838779956427
average: 0.6051633986928104


In [11]:
clf = SVC(kernel='rbf', random_state=0, C=2.2229964825261956)
roc_score(clf, 'Genomic (test) C=2.22', G_X, G_Y, Gt_X, Gt_Y)

Genomic (test) C=2.22 ROC AUC:  0.5681766186227407


0.5681766186227407

# MLP

In [12]:
scores = np.zeros(5)
for i, filename in enumerate(glob.glob('CV/MLP/*clinical*.txt')):
    with open(filename, 'r') as file:
        p_star, p_opt = file.readline().split()
        clf = MLPClassifier(solver='sgd', alpha=float(p_star), hidden_layer_sizes=(70), random_state=43)
        scores[i] = roc_score(clf, 'Clinical (validation) lambda='+p_star, Ctr_X, Ctr_Y, Cval_X, Cval_Y)
print('average:', np.mean(scores))

Clinical (validation) lambda=5.872786613189477 ROC AUC:  0.7004357298474946
Clinical (validation) lambda=2.2219468609395236 ROC AUC:  0.6827886710239652
Clinical (validation) lambda=3.9171014908092605 ROC AUC:  0.6827886710239652
Clinical (validation) lambda=12.17382727739662 ROC AUC:  0.6659041394335512
Clinical (validation) lambda=3.0721129988617593 ROC AUC:  0.6827886710239652
average: 0.6829411764705883


In [13]:
clf = MLPClassifier(solver='sgd', alpha=5.872786613189477, hidden_layer_sizes=(70), random_state=43)
roc_score(clf, 'Clinical (test) lambda=5.872', C_X, C_Y, Ct_X, Ct_Y)

Clinical (test) lambda=5.872 ROC AUC:  0.653854953099977


0.653854953099977

In [14]:
scores = np.zeros(5)
for i, filename in enumerate(glob.glob('CV/MLP/*genomic*.txt')):
    with open(filename, 'r') as file:
        p_star, p_opt = file.readline().split()
        clf = MLPClassifier(solver='sgd', alpha=float(p_opt), hidden_layer_sizes=(70), random_state=43)
        scores[i] = roc_score(clf, 'Genomic (validation) lambda='+p_star, Gtr_X, Gtr_Y, Gval_X, Gval_Y)
print('average:', np.mean(scores))

Genomic (validation) lambda=1.2648552168552958e-05 ROC AUC:  0.6264705882352941
Genomic (validation) lambda=1.2648552168552958e-05 ROC AUC:  0.6264705882352941
Genomic (validation) lambda=1.2648552168552958e-05 ROC AUC:  0.6264705882352941
Genomic (validation) lambda=1.2648552168552958e-05 ROC AUC:  0.6264705882352941
Genomic (validation) lambda=2.0235896477251556e-05 ROC AUC:  0.6264705882352941
average: 0.6264705882352941


In [15]:
clf = MLPClassifier(solver='sgd', alpha=1.2648552168552958e-05, hidden_layer_sizes=(70), random_state=43)
roc_score(clf, 'Genomic (test) alpha=1.26e-05', G_X, G_Y, Gt_X, Gt_Y)

Genomic (test) alpha=1.26e-05 ROC AUC:  0.578586135895676


0.578586135895676

# Random Forest

In [16]:
scores = np.zeros(5)
for i, filename in enumerate(glob.glob('CV/RandomForest/*clinical*.txt')):
    with open(filename, 'r') as file:
        p_star, p_opt = file.readline().split()
        clf = RandomForestClassifier(n_estimators=int(p_star), criterion='entropy', random_state=43)
        scores[i] = roc_score(clf, 'Clinical (validation) n_estimators='+p_star, Ctr_X, Ctr_Y, Cval_X, Cval_Y)
print('average:', np.mean(scores))

Clinical (validation) n_estimators=199 ROC AUC:  0.6583877995642702
Clinical (validation) n_estimators=173 ROC AUC:  0.6735294117647058
Clinical (validation) n_estimators=200 ROC AUC:  0.6583877995642702
Clinical (validation) n_estimators=135 ROC AUC:  0.6676470588235294
Clinical (validation) n_estimators=200 ROC AUC:  0.6583877995642702
average: 0.6632679738562091


In [17]:
clf = RandomForestClassifier(n_estimators=173, criterion='entropy', random_state=43)
roc_score(clf, 'Genomic (test) n_estimators=173', C_X, C_Y, Ct_X, Ct_Y)

Genomic (test) n_estimators=173 ROC AUC:  0.5950583390528482


0.5950583390528482

In [18]:
clf = RandomForestClassifier(n_estimators=173, criterion='entropy', random_state=43)
roc_score(clf, 'Genomic (test) n_estimators=173', G_X, G_Y, Gt_X, Gt_Y)

Genomic (test) n_estimators=173 ROC AUC:  0.5205902539464654


0.5205902539464654

# Stretch: Elastic Net, alpha=1

In [26]:
scores = np.zeros(5)
for i, filename in enumerate(glob.glob('CV/RegLR_1/*clinical*.txt')):
    with open(filename, 'r') as file:
        p_star, p_opt = file.readline().split()
        clf = RegularizedLogisticRegression.LogisticRegressionClassifier(reg_coeff=float(p_star), l1_ratio=1)
        scores[i] = roc_score(clf, 'Clinical (validation) lambda='+p_star, Ctr_X, Ctr_Y, Cval_X, Cval_Y)
print("average:", np.mean(scores))

Clinical (validation) lambda=100.0 ROC AUC:  0.6650326797385621
Clinical (validation) lambda=3.0721129988617593 ROC AUC:  0.7450980392156862
Clinical (validation) lambda=12.17382727739662 ROC AUC:  0.6407407407407407
Clinical (validation) lambda=8.119844993184008 ROC AUC:  0.7240740740740741
Clinical (validation) lambda=9.547716114208066 ROC AUC:  0.6861655773420481
average: 0.6922222222222223


In [27]:
clf = RegularizedLogisticRegression.LogisticRegressionClassifier(reg_coeff=3.0721129988617593, l1_ratio=1)
roc_score(clf, 'Clinical (test) lambda=3.072', C_X, C_Y, Ct_X, Ct_Y)

Clinical (test) lambda=3.072 ROC AUC:  0.6158773735987187


0.6158773735987187

In [29]:
ranks = np.argsort(clf.beta)
for i in ranks:
    if clf.beta[i] != 0:
        print("%s had %f weight" % (key[i], clf.beta[i]))

INTCLUST_1 had -11011.480799 weight
COHORT_5 had -8289.927339 weight
INTCLUST_3 had -7379.974703 weight
THREEGENE_2 had -7046.003777 weight
THREEGENE_3 had -6456.089240 weight
HISTOLOGICAL_SUBTYPE_2 had -6015.328896 weight
INTCLUST_5 had -5990.397432 weight
HISTOLOGICAL_SUBTYPE_7 had -5276.173470 weight
ER_IHC_2 had -5144.016881 weight
HER2_SNP6_2 had -4236.642200 weight
INFERRED_MENOPAUSAL_STATE_0 had -4094.247374 weight
CLAUDIN_SUBTYPE_6 had -4009.810037 weight
BREAST_SURGERY_0 had -3903.893993 weight
COHORT_4 had -3622.762824 weight
CHEMOTHERAPY_0 had -3562.526584 weight
INTCLUST_9 had -2941.769140 weight
INTCLUST_8 had -2935.293974 weight
ER_IHC_0 had -2909.613284 weight
RADIO_THERAPY_1 had -2879.539958 weight
COHORT_2 had -2833.075180 weight
HISTOLOGICAL_SUBTYPE_8 had -1821.156978 weight
CELLULARITY_3 had -1638.161069 weight
LATERALITY_0 had -1427.555107 weight
HISTOLOGICAL_SUBTYPE_6 had -1170.630621 weight
BREAST_SURGERY_2 had -884.112994 weight
HER2_SNP6_3 had -795.052937 weight

In [82]:
scores = np.zeros(5)
for i, filename in enumerate(glob.glob('CV/RegLR_1/*genomic*.txt')):
    with open(filename, 'r') as file:
        p_star, p_opt = file.readline().split()
        clf = RegularizedLogisticRegression.LogisticRegressionClassifier(reg_coeff=float(p_star), l1_ratio=1)
        scores[i] = roc_score(clf, 'Genomic (validation) lambda='+p_star, Gtr_X, Gtr_Y, Gval_X, Gval_Y)
print("average:", np.mean(scores))

Clinical (validation) lambda=10.353218432956616 ROC AUC:  0.5962962962962963
Clinical (validation) lambda=5.872786613189477 ROC AUC:  0.657843137254902
Clinical (validation) lambda=16.83180353330955 ROC AUC:  0.5533769063180829
Clinical (validation) lambda=14.314589375234785 ROC AUC:  0.6233115468409586
Clinical (validation) lambda=10.353218432956616 ROC AUC:  0.5962962962962963
average: 0.6054248366013072


In [83]:
clf = RegularizedLogisticRegression.LogisticRegressionClassifier(reg_coeff=14.314589375234785, l1_ratio=1)
roc_score(clf, 'Genomic (test) lambda=14.31', G_X, G_Y, Gt_X, Gt_Y)

Genomic (test) lambda=14.31 ROC AUC:  0.5290551361244566


0.5290551361244566

# Sklearn: Elastic Net, alpha=1

In [30]:
scores = np.zeros(5)
for i, filename in enumerate(glob.glob('CV/SGD_1/*clinical*.txt')):
    with open(filename, 'r') as file:
        p_star, p_opt = file.readline().split()
        clf = SGDClassifier(alpha=float(p_star), loss='log', penalty='elasticnet', l1_ratio=1, shuffle=False, random_state=43)
        scores[i] = roc_score(clf, 'Clinical (validation) lambda='+p_star, Ctr_X, Ctr_Y, Cval_X, Cval_Y)
print("average:", np.mean(scores))

Clinical (validation) lambda=0.030367711180354574 ROC AUC:  0.6684095860566449
Clinical (validation) lambda=0.023816855519761582 ROC AUC:  0.6465141612200436
Clinical (validation) lambda=0.02582618760682675 ROC AUC:  0.6372549019607843
Clinical (validation) lambda=0.049370478528390035 ROC AUC:  0.6431372549019607
Clinical (validation) lambda=0.038720387818125535 ROC AUC:  0.6684095860566449
average: 0.6527450980392157


In [31]:
clf = SGDClassifier(alpha=0.030367711180354574, loss='log', penalty='elasticnet', l1_ratio=1, shuffle=False, random_state=43)
roc_score(clf, 'Clinical (test) lambda=3.072', C_X, C_Y, Ct_X, Ct_Y)

Clinical (test) lambda=3.072 ROC AUC:  0.6380690917410203


0.6380690917410203

In [33]:
ranks = np.argsort(clf.coef_[0])
for i in ranks:
    if clf.coef_[0][i] != 0:
        print("%s had %f weight" % (key[i], clf.coef_[0][i]))

THREEGENE_1 had -0.199269 weight
BREAST_SURGERY_0 had -0.094551 weight
INTCLUST_3 had -0.073181 weight
INTCLUST_2 had 0.039887 weight
INTCLUST_6 had 0.144371 weight
NPI had 0.294165 weight
LYMPH_NODES_EXAMINED_POSITIVE had 0.382341 weight


In [22]:
scores = np.zeros(5)
for i, filename in enumerate(glob.glob('CV/SGD_1/*genomic*.txt')):
    with open(filename, 'r') as file:
        p_star, p_opt = file.readline().split()
        clf = SGDClassifier(alpha=float(p_star), loss='log', penalty='elasticnet', l1_ratio=1, shuffle=False, random_state=43)
        scores[i] = roc_score(clf, 'Genomic (validation) lambda='+p_star, Gtr_X, Gtr_Y, Gval_X, Gval_Y)
print("average:", np.mean(scores))

Genomic (validation) lambda=0.0016446761779946644 ROC AUC:  0.5711328976034858
Genomic (validation) lambda=0.008309941949353396 ROC AUC:  0.5896514161220043
Genomic (validation) lambda=0.013509935211980266 ROC AUC:  0.6014161220043572
Genomic (validation) lambda=0.004008806328898464 ROC AUC:  0.572004357298475
Genomic (validation) lambda=6.985879746785249e-05 ROC AUC:  0.6183006535947713
average: 0.5905010893246188


In [23]:
clf = SGDClassifier(alpha=6.985879746785249e-05, loss='log', penalty='elasticnet', l1_ratio=1, shuffle=False, random_state=43)
roc_score(clf, 'Genomic (test) lambda=3.072', G_X, G_Y, Gt_X, Gt_Y)

Genomic (test) lambda=3.072 ROC AUC:  0.5872797986730726


0.5872797986730726

In [21]:
sum(Cval_Y)/len(Cval_Y)

0.38848920863309355

In [23]:
sum(Ct_Y)/len(Ct_Y)

0.3357142857142857